In [ ]:
#default_exp solution

# Solution

This class defines solutions to instances of the Problem class. They usually result from calling a topo_solver with a problem object, but can also be instantiated manually by passing a problem and a density distribution.

In [ ]:
#exporti
import copy
import torch
import numpy as np
from typing import Union

from dl4to.solution import PlottingForSolution

In [ ]:
#hide
from nbdev.showdoc import show_doc

In [ ]:
#export
class Solution:
    """
    A class that defines solution objects.
    """
    def __init__(self, 
                 problem:"dl4to.problem.Problem", # The problem to which this is a solution.
                 θ:torch.Tensor, # The tensor that defines a density distribution that solves the TO problem.
                 name:str=None, # The name of the solution.
                 enforce_θ_on_Ω_design:bool=True # Whether the density distribution should be modified such that it fulfills the density restrictions imposed by the problem object.
                ): 
        self.enforce_θ_on_Ω_design = enforce_θ_on_Ω_design
        self.name = name
        self._problem = problem
        self.θ = θ
        self._check_θ_shape_and_range()


    @property
    def θ(self):
        return self._θ


    @θ.setter
    def θ(self, θ_new):
        self._θ = θ_new
        if self.enforce_θ_on_Ω_design:
            Ω_design = self.problem.Ω_design.to(θ_new.device)
            self._θ = torch.where(Ω_design == -1., self.θ, Ω_design.type(self.θ.dtype))
        self.u = None
        self.u_binary = None


    @property
    def pde_solver(self):
        return self.problem.pde_solver


    @pde_solver.setter
    def pde_solver(self, new_pde_solver):
        self.problem._pde_solver = new_pde_solver
        self.u = None
        self.u_binary = None


    @property
    def dtype(self):
        return self.θ.dtype


    @dtype.setter
    def dtype(self, dtype):
        self.problem.dtype = dtype
        self.θ = self.θ.type(dtype)


    @property
    def problem(self):
        return self._problem


    @property
    def shape(self):
        return self.problem.shape


    @property
    def device(self):
        return self.θ.device


    @device.setter
    def device(self, device):
        self.θ = self.θ.to(device)
        if self.u is not None:
            self.u = self.u.to(device)
        if self.u_binary is not None:
            self.u_binary = self.u_binary.to(device)


    def to(self, device):
        """
        Moves the solution object to `device`.
        """
        self.device = device
        return self


    def get_θ(self, 
              binary:bool=False # Whether the density should be binarized, i.e., thresholded at 0.5
             ):
        """
        Returns the density distribution `θ` from the solution object. Note that you can also obtain that density via `Solution.θ`, however this function has the option to return
        a binarized version of it.
        """
        if binary:
            θ_round = torch.zeros_like(self.θ)
            θ_round[self.θ > .5] = 1.
            return θ_round
        return self.θ


    def clone(self):
        """
        Returns a deepcopy of the Solution object.
        """
        return Solution(problem=self.problem,
                        θ=self.θ.clone(),
                        name=self.name,
                        enforce_θ_on_Ω_design=self.enforce_θ_on_Ω_design)


    def detach(self):
        """
        Returns a clone of the solution object that has a density θ which is detached from its computational graph.
        """
        detached_solution = self.clone()
        detached_solution._θ = self._θ.detach()
        return detached_solution


    def detach_(self):
        """
        Detaches θ from its computational graph. This is an in-place version of the `detach()` method.
        """
        self._θ.detach_()


    def plot(
        self,
        binary:bool=False, # Whether the density should be binarized before plotting and before solving the PDE. Note that the PDE is solved only if `solve_pde=True` and problem has a PDE solver attached to it.
        solve_pde:bool=False, # Whether to solve the PDE of linear elasticity and plot the absolute displacements and von Mises stresses.
        normalize_σ_vm:bool=True, # Whether the von Mises stresses should be normalized with the yield stress.
        threshold:float=0., # Density threshold below which voxels should be displayed as empty.
        display:bool=True, # Whether the figure is displayed.
        file_path:str=None, # Path where the figure is saved.
        camera_position:tuple=(0,.1,.12), # x, y, and z coordinates of the camera position.
        show_design_space:bool=False, # Whether to highlight the voxels that have a design space information of -1 assigned to them.
        use_pyvista:bool=False, # Whether to use pyvista for plotting. If `False`, then plotly is used. Pyvista generates better looking visualizations, but does not support basic features like colorbars, title display and saving.
        window_size:Union[tuple,list]=(800,800), # The size of the window that displays the plot. Only has an effect if `use_pyvista=True`.
        smooth_iters:int=0, # The number of smoothing iterations for better looking visualizations. Only has an effect if `use_pyvista=True`.
        show_colorbar:bool=True, # Determines whether a reference colorbar is displayed for the plotted voxel color values.
        show_axislabels:bool=False, # Whether the 3d axes are labelled with their dimensions.
        show_ticklabels:bool=False, # Whether the 3d axes ticks are displayed and labeled.
        export_png:bool=False # Whether the figure is exported and saved as a png file, in addition to the standard html format.
    ):
        """
        Renders a 3D figure that displays the density distribution and, optionally, additional ones that display the displacements and von Mises stresses for that density.
        """
        PlottingForSolution()(
            solution=self,
            binary=binary,
            solve_pde=solve_pde,
            normalize_σ_vm=normalize_σ_vm,
            threshold=threshold,
            display=display, 
            file_path=file_path, 
            camera_position=camera_position, 
            show_design_space=show_design_space,
            use_pyvista=use_pyvista,
            window_size=window_size,
            smooth_iters=smooth_iters,
            show_colorbar=show_colorbar,
            show_axislabels=show_axislabels, 
            show_ticklabels=show_ticklabels, 
            export_png=export_png
        )


    def solve_pde(self, 
                  p:float=1., # Denotes the SIMP exponent and should usually be left at its default value of 1.
                  binary:bool=False # Whether the density should be binarized before solving the PDE.
                 ):
        """
        Solves the PDE of linear elasticity for the current solution. Returns the displacement tensor, stress tensor and the von Mises stress tensor.
        """
        if self.pde_solver is None:
            raise AttributeError("solution.problem has no PDE solver attached to it.")
        u, σ, σ_vm = self.pde_solver(self, p=p, binary=binary)
        u = u.to(self.θ.device)
        σ = σ.to(self.θ.device)
        σ_vm = σ_vm.to(self.θ.device)
        return u, σ, σ_vm


    def _check_θ_shape_and_range(self, θ=None):
        if θ is None:
            θ = self.θ

        if θ is None:
            return

        if not isinstance(θ, torch.Tensor):
            raise TypeError("θ must be None or a torch.Tensor")

        if len(θ.shape) != 4 or θ.shape[0] != 1:
            print(θ.shape)
            raise ValueError("θ tensor is not of the right shape.")

        if torch.any(θ < 0) or torch.any(1 < θ):
            raise ValueError("θ tensor contrains values outside the interval [0, 1].")

        if "Problem" not in str(self.problem):
            raise TypeError("problem must be an Problem")

        if θ is not None:
            if not np.all(θ.shape[1:] == self.shape):
                raise ValueError("θ does not fit to the associated Problem.")


    def eval(self, 
             criterion:"dl4to.criteria.Criterion" # The criterion for which the solution should be evaluated.
            ):
        """
        Evaluate the solution object with `criterion`. Only works for unsupervised criteria. Returns a `torch.Tensor`.
        """
        assert ~criterion.supervised, print("`solution.eval()` only works for unsupervised criteria.")
        return criterion([self])


    def __rmul__(self, 
                 λ:float # The multiplier for the density.
                ):
        """
        Multiplication of a solution with a scalar returns a clone of the solution that has the original density distribution that is rescaled with `λ`. Returns a new `solution` object.
        """
        solution = self.clone()
        solution.θ = λ * self.θ
        return solution


    def __mul__(self, 
                λ:float # The multiplier for the density.
               ):
        """
        Multiplication of a solution with a scalar returns a clone of the solution that has the original density distribution that is rescaled with `λ`. Returns a new `solution` object.
        """
        return self.__rmul__(λ)

## Methods

In [ ]:
show_doc(Solution.get_θ)

<h4 id="Solution.get_θ" class="doc_header"><code>Solution.get_θ</code><a href="__main__.py#L77" class="source_link" style="float:right">[source]</a></h4>

> <code>Solution.get_θ</code>(**`binary`**:`bool`=*`False`*)

Returns the density distribution `θ` from the solution object. Note that you can also obtain that density via [`Solution.θ`](/dl4tosolution.html#Solution.θ), however this function has the option to return
a binarized version of it.

||Type|Default|Details|
|---|---|---|---|
|**`binary`**|`bool`|`False`|Whether the density should be binarized, i.e., thresholded at 0.5|


In [ ]:
show_doc(Solution.clone)

<h4 id="Solution.clone" class="doc_header"><code>Solution.clone</code><a href="__main__.py#L91" class="source_link" style="float:right">[source]</a></h4>

> <code>Solution.clone</code>()

Returns a deepcopy of the Solution object.

In [ ]:
show_doc(Solution.detach)

<h4 id="Solution.detach" class="doc_header"><code>Solution.detach</code><a href="__main__.py#L99" class="source_link" style="float:right">[source]</a></h4>

> <code>Solution.detach</code>()

Returns a clone of the solution object that has a density θ which is detached from its computational graph.

In [ ]:
show_doc(Solution.detach_)

<h4 id="Solution.detach_" class="doc_header"><code>Solution.detach_</code><a href="__main__.py#L108" class="source_link" style="float:right">[source]</a></h4>

> <code>Solution.detach_</code>()

Detaches θ from its computational graph. This is an in-place version of the `detach()` method.

In [ ]:
show_doc(Solution.plot)

<h4 id="Solution.plot" class="doc_header"><code>Solution.plot</code><a href="__main__.py#L115" class="source_link" style="float:right">[source]</a></h4>

> <code>Solution.plot</code>(**`binary`**:`bool`=*`False`*, **`solve_pde`**:`bool`=*`False`*, **`normalize_σ_vm`**:`bool`=*`True`*, **`threshold`**:`float`=*`0.0`*, **`display`**:`bool`=*`True`*, **`file_path`**:`str`=*`None`*, **`camera_position`**:`tuple`=*`(0, 0.1, 0.12)`*, **`show_design_space`**:`bool`=*`False`*, **`use_pyvista`**:`bool`=*`False`*, **`window_size`**:`Union`\[`tuple`, `list`\]=*`(300, 300)`*, **`smooth_iters`**:`int`=*`0`*, **`show_colorbar`**:`bool`=*`True`*, **`show_axislabels`**:`bool`=*`False`*, **`show_ticklabels`**:`bool`=*`False`*, **`export_png`**:`bool`=*`False`*)

Renders a 3D figure that displays the density distribution and, optionally, additional ones that display the displacements and von Mises stresses for that density.

||Type|Default|Details|
|---|---|---|---|
|**`binary`**|`bool`|`False`|Whether the density should be binarized before plotting and before solving the PDE. Note that the PDE is solved only if `solve_pde=True` and problem has a PDE solver attached to it.|
|**`solve_pde`**|`bool`|`False`|Whether to solve the PDE of linear elasticity and plot the absolute displacements and von Mises stresses.|
|**`normalize_σ_vm`**|`bool`|`True`|Whether the von Mises stresses should be normalized with the yield stress.|
|**`threshold`**|`float`|`0.0`|Density threshold below which voxels should be displayed as empty.|
|**`display`**|`bool`|`True`|Whether the figure is displayed.|
|**`file_path`**|`str`|`None`|Path where the figure is saved.|
|**`camera_position`**|`tuple`|`(0, 0.1, 0.12)`|x, y, and z coordinates of the camera position.|
|**`show_design_space`**|`bool`|`False`|Whether to highlight the voxels that have a design space information of -1 assigned to them.|
|**`use_pyvista`**|`bool`|`False`|Whether to use pyvista for plotting. If `False`, then plotly is used. Pyvista generates better looking visualizations, but does not support basic features like colorbars, title display and saving.|
|**`window_size`**|`typing.Union[tuple, list]`|`(300, 300)`|The size of the window that displays the plot. Only has an effect if `use_pyvista=True`.|
|**`smooth_iters`**|`int`|`0`|The number of smoothing iterations for better looking visualizations. Only has an effect if `use_pyvista=True`.|
|**`show_colorbar`**|`bool`|`True`|Determines whether a reference colorbar is displayed for the plotted voxel color values.|
|**`show_axislabels`**|`bool`|`False`|Whether the 3d axes are labelled with their dimensions.|
|**`show_ticklabels`**|`bool`|`False`|Whether the 3d axes ticks are displayed and labeled.|
|**`export_png`**|`bool`|`False`|Whether the figure is exported and saved as a png file, in addition to the standard html format.|


In [ ]:
show_doc(Solution.solve_pde)

<h4 id="Solution.solve_pde" class="doc_header"><code>Solution.solve_pde</code><a href="__main__.py#L156" class="source_link" style="float:right">[source]</a></h4>

> <code>Solution.solve_pde</code>(**`p`**:`float`=*`1.0`*, **`binary`**:`bool`=*`False`*)

Solves the PDE of linear elasticity for the current solution. Returns the displacement tensor, stress tensor and the von Mises stress tensor.

||Type|Default|Details|
|---|---|---|---|
|**`p`**|`float`|`1.0`|Denotes the SIMP exponent and should usually be left at its default value of 1.|
|**`binary`**|`bool`|`False`|Whether the density should be binarized before solving the PDE.|


In [ ]:
show_doc(Solution.eval)

<h4 id="Solution.eval" class="doc_header"><code>Solution.eval</code><a href="__main__.py#L193" class="source_link" style="float:right">[source]</a></h4>

> <code>Solution.eval</code>(**`criterion`**:`dl4to.criteria.Criterion`)

Evaluate the solution object with `criterion`. Only works for unsupervised criteria. Returns a `torch.Tensor`.

||Type|Default|Details|
|---|---|---|---|
|**`criterion`**|`dl4to.criteria.Criterion`||The criterion for which the solution should be evaluated.|


In [ ]:
show_doc(Solution.__mul__)

<h4 id="Solution.__mul__" class="doc_header"><code>Solution.__mul__</code><a href="__main__.py#L214" class="source_link" style="float:right">[source]</a></h4>

> <code>Solution.__mul__</code>(**`λ`**:`float`)

Multiplication of a solution with a scalar returns a clone of the solution that has the original density distribution that is rescaled with `λ`. Returns a new `solution` object.

||Type|Default|Details|
|---|---|---|---|
|**`λ`**|`float`||The multiplier for the density.|


# Examples

In [ ]:
#ignore
from dl4to.solution import Solution
from dl4to.datasets import BasicDataset

problem = BasicDataset(resolution=40).ledge()
shape = [40, 4, 8]
θ = torch.zeros(1, *shape)
solution_zeros = Solution(problem, θ)

In [ ]:
#ignore
camera_position = (0, 0.35, 0.2)
solution_zeros.plot(camera_position=camera_position,
                    display=False)

![zero_density](https://dl4to.github.io/dl4to/images/1_zero_density.png)

As we can see, the density distribution $\theta$ has been modified inside of the solution object such that it is not $0$ everywhere anymore. More precisely, it has been adjusted according to the design space information that we prescribed in the problem formulation: We enforced densities of $1$ at certain voxels by setting $\Omega_\text{dirichlet}$ to $1$. 

We can check that the density distribution inside of the solution object has indeed been modified and this is not just the visualization:

In [ ]:
#ignore
assert torch.any(solution_zeros.θ != 0)
assert torch.all(θ == 0)

The easiest type of solution is what we call a "trivial solution". The density distribution of a trivial solution in $1$ everywhere, where it is permitted (i.e. where $\Omega_\text{design}$ is not $0$). Together with the zero-density example above this therefore constitutes the simplest solution to a TO problem - we simply choose the thickest possible structure!

Each problem object directly comes with its own trivial solution. It can be accessed via:

In [ ]:
#ignore
trivial_solution = problem.trivial_solution

trivial_solution.plot(camera_position=camera_position,
                      display=False)

![trivial_density](https://dl4to.github.io/dl4to/images/1_trivial_density.png)

In order to evaluate the stresses we need to solve the partial differential equation (PDE) of linear elasticity. This library comes with its own in-built finite differences method (FDM) solver, which solves the PDE for us. We found handling with finite differences easier than with finite elements. This is attributed to the regular grid structure, which makes the FDM a suitable and intuitive approach. It is however also possible to include custom PDE solvers, e.g., learned PDE solvers - which we will discuss later.


PDE solvers are passed to problem instances:

In [ ]:
#ignore
from dl4to.pde import FDM

problem.pde_solver = FDM()

Passing PDE solvers to problems (instead of passing them to solutions or TO solvers) my seem unintuitive at first, but it comes with several advantages:
First, all solution objects that are derived from this problem will also have access to the PDE solver. The same holds for all TO solver algorithms that we apply to this problem. Second, our implementation automatically constructs most of the PDE system matrix in the background when it is passed to a problem. This saves a lot of time for all future evaluations.

We can now use this FDM solver to solve the PDE. This is done via the "solve_pde" command, which returns three tensors:
- The displacements $u$, which is a ($3\times n_x \times n_y \times n_z$)-tensor.
- The stresses $\sigma$, which is a symmetric ($9\times n_x \times n_y \times n_z$)-tensor.
- The von Mises stresses $\sigma_\text{vM}$, which is a ($1\times n_x \times n_y \times n_z$)-tensor, i.e., a scalar field.

In [ ]:
#ignore
u, σ, σ_vm = trivial_solution.solve_pde()

After the PDE has been solved for a solution, the displacements $u$ are stored inside the solution object and can be accessed via "trivial_solution.u". This avoids solving the same PDE several times.

In order to check if the von Mises stresses are too large, we need to compare its maximum to the yield stress $\sigma_\text{ys}$ of the material:

In [ ]:
#ignore
σ_vm.max() / problem.σ_ys

tensor(0.0198)

Since the fraction returns a value below $1$, this indicates that the structure indees holds the applied forces and does not break!

We can also visualize the spacial distribution of the (normed) displacements and von Mises stresses by passing "solve_pde=True" to the plotting function:

In [ ]:
#ignore
trivial_solution.plot(camera_position=camera_position,
                      solve_pde=True,
                      display=False)

![trivial_density_theta](https://dl4to.github.io/dl4to/images/1_trivial_density_theta.png)
![trivial_density_u](https://dl4to.github.io/dl4to/images/1_trivial_density_u.png)
![trivial_density_stress](https://dl4to.github.io/dl4to/images/1_trivial_density_stress.png)

In [ ]:
#hide
import os
from dl4to.pde import FDM
from dl4to.datasets import BasicDataset

In [ ]:
%%time
#hide

def test_that_get_θ_returns_θ_or_rounded_version():
    problem = BasicDataset().ledge()

    θ = torch.rand(1, *problem.shape)
    solution = Solution(problem, θ, enforce_θ_on_Ω_design=False)

    θ_round = θ.clone()
    θ_round[θ < .5] = 0.
    θ_round[.5 <= θ] = 1.

    assert torch.allclose(θ,       solution.get_θ(binary=False))
    assert torch.allclose(θ_round, solution.get_θ(binary=True))


test_that_get_θ_returns_θ_or_rounded_version()

CPU times: user 3.06 ms, sys: 0 ns, total: 3.06 ms
Wall time: 2.81 ms


In [ ]:
%%time
#hide

def test_can_one_differentiate_θ_in_Solution():
    problem = BasicDataset().ledge()

    θ_ = torch.rand(1, *problem.shape, requires_grad=True)
    θ = θ_ * .5
    solution = Solution(problem, θ)

    assert solution.get_θ().requires_grad

    grad = torch.autograd.grad(solution.get_θ().mean(), θ_)
    assert len(grad) == 1
    grad = grad[0]
    assert grad.shape == (1, *problem.shape)


test_can_one_differentiate_θ_in_Solution()

CPU times: user 0 ns, sys: 3.83 ms, total: 3.83 ms
Wall time: 3.43 ms


In [ ]:
%%time
#hide

def test_move_to_device(device='cpu'):
    problem = BasicDataset().ledge()

    θ = torch.rand(1, *problem.shape)
    solution = Solution(problem, θ)
    solution.device = device

    assert str(solution.get_θ().device) == device


test_move_to_device()

CPU times: user 1.45 ms, sys: 88 µs, total: 1.54 ms
Wall time: 1.37 ms


In [ ]:
%%time
#hide

def test_that_the_clone_has_the_same_attributes():
    problem = BasicDataset().ledge()

    θ = torch.rand(1, *problem.shape)
    solution = Solution(problem, θ)
    solution_clone = solution.clone()

    assert torch.all(solution.get_θ() == solution_clone.get_θ())

    # only basic None checking for object-attributes
    if (solution.problem is None) or (solution_clone.problem is None):
        assert (solution.problem is None) and (solution_clone.problem is None)
    if (solution.pde_solver is None) or (solution_clone.pde_solver is None):
        assert (solution.pde_solver is None) and (solution_clone.pde_solver is None)


test_that_the_clone_has_the_same_attributes()

CPU times: user 2.4 ms, sys: 0 ns, total: 2.4 ms
Wall time: 2.23 ms


In [ ]:
%%time
#hide

def test_that_thresholding_θ_has_an_effect():
    problem = BasicDataset().ledge()

    θ = torch.rand(1, *problem.shape)
    solution = Solution(problem, θ=θ, enforce_θ_on_Ω_design=False)
    solution_clone = solution.clone()

    solution.θ = torch.rand(1, *problem.shape)

    assert torch.all(solution.get_θ() != solution_clone.get_θ()), "densities not detached"

    solution._problem = 'problem_placeholder'
    solution._pde_solver = 'pde_solver_placeholder'

    assert solution_clone.pde_solver is None, "pde_solver is not none"


test_that_thresholding_θ_has_an_effect()

CPU times: user 3.64 ms, sys: 0 ns, total: 3.64 ms
Wall time: 3.18 ms


In [ ]:
%%time
#hide

def test_that_the_clone_has_decuplet_attributes_():
    problem = BasicDataset().ledge()

    θ = torch.rand(1, *problem.shape)
    solution = Solution(problem, θ=θ, enforce_θ_on_Ω_design=False)
    solution_clone = solution.clone()

    solution.θ = torch.rand(1, *problem.shape)

    assert torch.all(solution.get_θ() != solution_clone.get_θ())

    solution._problem = 'problem_placeholder'
    solution._pde_solver = 'pde_solver_placeholder'

    assert solution_clone.pde_solver is None


test_that_the_clone_has_decuplet_attributes_()

CPU times: user 2.23 ms, sys: 0 ns, total: 2.23 ms
Wall time: 2.06 ms


In [ ]:
%%time
#hide

def test_u_and_σ_and_σ_vm_shapes():
    problem = BasicDataset(resolution=15).ledge()
    θ = .1 + torch.rand(1, *problem.shape) * .8
    θ[:, 0] = 0
    problem.pde_solver = FDM()
    solution = Solution(problem, θ=θ)
    u, σ, σ_vm = solution.solve_pde()

    assert u.shape[0] == 3
    assert u.shape[1:] == problem.shape
    assert σ.shape[0] == 9
    assert σ.shape[1:] == problem.shape
    assert σ_vm.shape[0] == 1
    assert σ_vm.shape[1:] == problem.shape

test_u_and_σ_and_σ_vm_shapes()

CPU times: user 1.9 s, sys: 51.1 ms, total: 1.95 s
Wall time: 483 ms
